In [2]:
from goatools.obo_parser import GODag
godag = GODag("resource_data/go-basic.obo")

resource_data/go-basic.obo: fmt(1.2) rel(2020-01-01) 47,337 GO Terms


In [ ]:
import os
from goatools.associations import dnld_assc
#fin_gaf = os.path.join(os.getcwd(), "goa_uniprot_all.gaf")
#associations = dnld_assc(fin_gaf, godag, namespace='all')

In [3]:
from goatools.anno.idtogos_reader import IdToGosReader
associations = IdToGosReader("resource_data/associations-uniprot-sp-20200116.tab", godag=godag).get_id2gos('all')

HMS:0:00:19.736018 3,571,800 annotations READ: resource_data/associations-uniprot-sp-20200116.tab 
536442 IDs in loaded association branch, all


In [0]:
go_id1 = 'GO:0048364'
go_id2 = 'GO:0032501'

In [4]:
from goatools.semantic import deepest_common_ancestor
from goatools.semantic import TermCounts, get_info_content


In [0]:
from goatools.semantic import semantic_similarity

sim = semantic_similarity(go_id1, go_id2, godag)
print('The semantic similarity between terms {} and {} is {}.'.format(go_id1, go_id2, sim))

In [10]:
from goatools.semantic import TermCounts, get_info_content

# First get the counts of each GO term.
termcounts = TermCounts(godag, associations)

# Calculate the information content
# go_id = "GO:0008152"
# infocontent = get_info_content(go_id, termcounts)
# print('Information content ({}) = {}'.format(go_id, infocontent))

In [0]:
from goatools.semantic import lin_sim

sim_l = lin_sim(go_id1, go_id2, godag, termcounts)
print('Lin similarity score ({}, {}) = {}'.format(go_id1, go_id2, sim_l))

In [0]:
freq = termcounts.get_term_freq(go_id)
freq

In [5]:
import multiprocessing

def BMA(GO_list1,GO_list2,termcounts,similarity_method=None):
    summationSet12 = 0.0
    summationSet21 = 0.0
    for id1 in GO_list1:
        similarity_values = []
        for id2 in GO_list2:
            similarity_values.append(Rel_Metric(id1,id2,godag,termcounts))
        summationSet12 += max(similarity_values)
    for id2 in GO_list2:
        similarity_values = []
        for id1 in GO_list1:
            similarity_values.append(Rel_Metric(id2,id1,godag,termcounts))
        summationSet21 += max(similarity_values)
    return (summationSet12+summationSet21)/(len(GO_list1)+len(GO_list2))

In [18]:
from goatools.gosubdag.gosubdag import GoSubDag


In [38]:
def get_highest_ic_anc(id,termcounts):
    gosubdag_r0 = GoSubDag([id], godag, prt=None)
    P = gosubdag_r0.rcntobj.go2parents[id]
    max_ic = 0
    for i in P:
        ic = get_info_content(i, termcounts)
        if(max_ic < ic):
            max_ic = ic
    return max_ic

In [35]:
def Rel_Metric(id1, id2, godag, termcounts):
    if id1 == '' or id2 == '':
        return -1
    goterm1 = godag[id1]
    goterm2 = godag[id2]
    if goterm1.namespace == goterm2.namespace:
        mica_goid = deepest_common_ancestor([id1, id2], godag)
        freq = termcounts.get_term_freq(mica_goid)
        info_content = get_info_content(mica_goid, termcounts)
        info_content1 = get_info_content(id1, termcounts)
        info_content2 = get_info_content(id2, termcounts)
        if(info_content1 == 0):
             info_content1 = get_highest_ic_anc(id1,termcounts)
        if(info_content2 == 0):
             info_content2 = get_highest_ic_anc(id2,termcounts)
        return (2 * info_content * (1 - freq)) / (info_content1 + info_content2)
    else:
        return -1

In [0]:
def read_input(file_path):

  """"
  This function reads a csv with two columns of GO terms, coming from two datasets
  Returns two lists of GO terms
  """

  GO_list1, GO_list2 = list(), list()
  
  with open(file_path, "r", encoding='utf-8-sig') as in_f: #other files might require other encoding?
    for line in in_f:
      if not line.startswith("GO"): # skip header
        continue
      print(line)
      GO1, GO2, _ = line.strip().split(";", maxsplit=2)
      
      # add GO1 to GO_list1
      GO1_sub = list()
      if "," in GO1:
        mGO = GO1.split(",")
        for GO in mGO:
          GO1_sub.append(GO)
      else:
        GO1_sub.append(GO1)
      GO_list1.append(GO1_sub)
      

      # add GO2 to GO_list2
      GO2_sub = list()
      if "," in GO2:
        mGO = GO2.split(",")
        for GO in mGO:
          GO2_sub.append(GO)
      else:
        GO2_sub.append(GO2)
      GO_list2.append(GO2_sub)
    
    return GO_list1, GO_list2

In [0]:
GO_list1, GO_list2 = read_input('GO terms.csv')

In [0]:
print(GO_list1)
print(GO_list2)

In [0]:
for i in range(0,len(GO_list1)):
  BMA_test = BMA(GO_list1[i],GO_list2[i])
  print(BMA_test)

In [0]:
termcounts = TermCounts(godag, associations)

In [0]:
for i in range(0,len(GO_list1)):
  BMA_test = BMA(GO_list1[i],GO_list2[i],termcounts)
  print(BMA_test)

0.33877743862689497
0.16645336402284938
0.0
-1.0
0.4958756327741224
0.11295263352259761
0.09699230064168216
0.3038198818835536
-1.0
0.0


In [40]:
BMA_test = BMA(['GO:0005488'],['GO:0048037','GO:0036094'],termcounts)
print(BMA_test)

0.11295263352259761


In [34]:
get_highest_ic_anc('GO:0090634')

2.3339909262250016
3.656836456801969
-0.0


3.656836456801969

In [32]:
Rel_Metric()

ImportError: cannot import name 'get_go2parents'